In [282]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [283]:
import os
import os.path
import pandas as pd
import numpy as np
import sys
import re
import sys
import json
from datetime import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

pd.set_option('display.max_columns', 50)

In [284]:
from cig_script.config.config_env import Config

# from cig_script.mail_copy_input_format import _get_one_mail_dict
# from cig_script.mail_copy_input_format import _get_mul_mail_dict
# from cig_script.mail_copy_input_format import _get_all_sheet_from_mail
# from cig_script.mail_copy_input_format import _check_include_df

# from cig_script.mail_copy_input_format import _autowidth_excel
# from datetime import datetime





In [285]:
tot_excel_from_mail = Config().tot_excel_from_mail
tot_mail_sheet = Config().total_mail_sheet_name
total_sheet_df = pd.read_excel(tot_excel_from_mail,sheet_name=tot_mail_sheet)

In [286]:

total_sheet_df

,CONSIGNEE NAME,MODEL,YR,CHASSINO,ACID NO,IMPORTER TAX NUMBER,FREIGHT FORWARDER ID,SHEET,gen_time,EDI_NO,H_BL_NO
0,SALEH AHMED MOHAMED SALEH,K3,2013,KNAFK412BDA050028,9992022070000061164,999999999,950449415,ELMONGY,2022-09-01 22:54,CIGSINPS2208F002,CIGSINPS2208F002
1,NABIELAH MAGDY AATTIA MOHAMED AATTIA HELAL,AVANTE,2010,KMHDU41BBAU098545,9992022070000038337,999999999,940426900,ELMONGY,2022-09-01 22:54,CIGSINPS2208F002,CIGSINPS2208F005
2,BASSIOUNY MOHAMED HAFEZ MOHAMED ELSHAFAAY,FORTE,2009,KNAFH22139A014476,9992022070000061172,999999999,950449415,ELMONGY,2022-09-01 22:54,CIGSINPS2208F002,CIGSINPS2208F009
3,REDA MAHMOUD SADEK MOUSTAFA,AVANTE,2016,KMHD741DBGU173749,9992022080000014089,999999999,903204053,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F004
4,AASEM MOHAMED ABD ELFATAH ABD ELRAHMAN,K3,2013,KNAFK412BDA083247,9992022080000014329,999999999,903204053,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F011
5,AAWAAD ABD ELAATY MOHAMED EBRAHIM,TRAX,2016,KLABA76BDGB748116,9992022080000014402,999999999,903204053,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F012
6,MOHAMED SHAABAN KHALF HUSSIEN,FORTE,2009,KNAFH22139A050074,9992022080000014485,999999999,903204053,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F013
7,MOHAMED BAHAG ELDIEN AHMED HUSSIEN,AVANTE,2011,KMHDG41DBBU137819,9992022080000014667,999999999,903204053,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F014
8,SHAABAN ELDESSOUKY MOHAMED WAHBA,AVANTE,2007,KMHDU41BP7U018874,9992022080000028923,999999999,950496286,YOUSEF,2022-09-01 22:54,CIGSINPS2208F004,CIGSINPS2208F015
9,AHMED MOHAMED ELAAWADY ABD ELAAL,AVANTE,2011,KMHDG41DBBU024372,9992022070000061545,999999999,950496286,SAM,2022-09-01 22:54,CIGSINPS2208F007,CIGSINPS2208F007


In [287]:
edi_path = Config().edi_data_path
edi_sheet = Config().edi_sheet

In [288]:
edi_df = pd.read_excel(edi_path,sheet_name=edi_sheet)
edi_df

,EDI NO.,H.BL NO,M.BL NO,MODEL,YR,CHASSINO.,WEIGHT,CBM,ACID NO.
0,CIGSINPS2208F001,CIGSINPS2208F001,NaN,AVANTE MD,2012,KMHDG41DBCU448663,1150,10.482,2049431672022060010
1,CIGSINPS2208F001,CIGSINPS2208F001,NaN,AVANTE HD,2007,KMHDU41BP7U302311,1150,10.482,2049431672022060010
2,CIGSINPS2208F001,CIGSINPS2208F001,NaN,K3,2013,KNAFK412BDA011679,1660,11.648,2049431672022060010
3,CIGSINPS2208F001,CIGSINPS2208F001,NaN,AVANTE MD,2013,KMHDG41DBDU665209,1165,10.482,2049431672022060010
4,CIGSINPS2208F001,CIGSINPS2208F001,NaN,FORTE,2012,KNAFU415BCA951875,1350,11.739,2049431672022060010
...,...,...,...,...,...,...,...,...,...
419,CIGSINPS2208F008,CIGSINPS2208F008,NaN,FORTE,2010,KNAFT411BAA233766,1000,11.739,2049076832022080087
420,CIGSINPS2208F008,CIGSINPS2208F008,NaN,FORTE,2010,KNAFU411BAA263508,1000,11.739,2049076832022080087
421,CIGSINPS2208F008,CIGSINPS2208F008,NaN,FORTE,2009,KNAFH22139A070842,1000,11.739,2049076832022080087
422,CIGSINPS2208F008,CIGSINPS2208F008,NaN,AVANTE,2012,KMHDG41DBCU277583,1000,10.482,2049076832022080087


In [292]:
total_sheet_df[['EDI_NO','H_BL_NO']] = np.nan
edi_no_list = edi_df['EDI NO.'].drop_duplicates().values

for row in total_sheet_df.CHASSINO.iteritems():
    idx,val = row
    mul_chassino = {}
    one_chassino = ''
    edi_no = ''
    
    try:
        mul_chassino = json.loads(val)   #mul car one bl
    except:
        one_chassino = val
    
    if one_chassino:
        try:
            edi_no = edi_df.loc[(edi_df['CHASSINO.'] == val),'EDI NO.'].values[0]
        except:
            print('there is no matched EDI NUM for : ', val)
            pass        
        
    if mul_chassino:
        key_count = len(mul_chassino)
        first_key = next(iter(mul_chassino))
        mul_chassino_first_value = mul_chassino[first_key]
        
        try:
            edi_no = edi_df.loc[(edi_df['CHASSINO.'] == mul_chassino_first_value),'EDI NO.'].values[0]
        except:
            print('there is no matched EDI NUM for : ', mul_chassino_first_value)
            pass        
    
    
    result = re.search('(\d+)\s?$',edi_no)
    
    if result == None:
        pass
    else:
        bl_num = int(result.group(1))
        hbl_no_sr = total_sheet_df['H_BL_NO'] 
        edi_match_sr = total_sheet_df.loc[(total_sheet_df['EDI_NO']==edi_no),'H_BL_NO']
        
        if edi_match_sr.dropna().empty:
            _h_bl_no = edi_no
        else:
            hbl_no_max = hbl_no_sr.dropna().max()
            
            if not hbl_no_max in edi_no_list:
                max_result = re.search('(\d+)\s?$',hbl_no_max)
                max_val = max_result.group(1)
                max_prefix = hbl_no_max.split(max_val)[0]
                max_val = int(max_val) + 1
                hbl_no_max = max_prefix + str(max_val).zfill(3)
            _h_bl_no = hbl_no_max
            
            while hbl_no_max in edi_no_list:
                max_result = re.search('(\d+)\s?$',hbl_no_max)
                max_val = max_result.group(1)
                max_prefix = hbl_no_max.split(max_val)[0]
                max_val = int(max_val) + 1
                hbl_no_max = max_prefix + str(max_val).zfill(3)
            _h_bl_no = hbl_no_max
                
            
        
        total_sheet_df.loc[idx,'EDI_NO'] = edi_no
        total_sheet_df.loc[idx,'H_BL_NO'] = _h_bl_no
print(total_sheet_df[['CHASSINO','EDI_NO','H_BL_NO']])

    
    

there is no matched EDI NUM for :  KMHJ5812GLU324020
there is no matched EDI NUM for :  KPBXHAR1FP019286
                                             CHASSINO            EDI_NO  \
0                                   KNAFK412BDA050028  CIGSINPS2208F002   
1                                   KMHDU41BBAU098545  CIGSINPS2208F002   
2                                   KNAFH22139A014476  CIGSINPS2208F002   
3                                   KMHD741DBGU173749  CIGSINPS2208F004   
4                                   KNAFK412BDA083247  CIGSINPS2208F004   
5                                   KLABA76BDGB748116  CIGSINPS2208F004   
6                                   KNAFH22139A050074  CIGSINPS2208F004   
7                                   KMHDG41DBBU137819  CIGSINPS2208F004   
8                                   KMHDU41BP7U018874  CIGSINPS2208F004   
9                                   KMHDG41DBBU024372  CIGSINPS2208F007   
10                                  KNAFZ412BDA004986  CIGSINPS2208F00

In [273]:
total_sheet_df
for col in total_sheet_df:
    print(col)
# for idx,row in total_sheet_df.iterrows():
#     print(row)

YR
CONSIGNEE NAME
MODEL
CHASSINO
ACID NO
IMPORTER TAX NUMBER
FREIGHT FORWARDER ID
SHEET
gen_time
EDI_NO
H_BL_NO


In [241]:
edi_match_sr = total_sheet_df['H_BL_NO']
edi_match_sr.dropna().max()

'CIGSINPS2208F002'